# Chapter 21: Asynchronous Programming

> The problem with normal approaches to asynchronous programming is that they're all-or-nothing propositions. You rewrite all your code so none of it blocks or you're just wasting your time. 
> -- Alvaro Videla and Jason J. W. Williams, RabbitMQ in Action

This chapter addresses three major topics that are closely related:

- Python's `async def`, `await`, `async with`, and `async for` constructs
- Objects supporting those constructs: native coroutines and asynchronous variants of context managers, iterables, generators, and comprehensions
- `asyncio` and other asynchronous libraries

## A Few Definitions

- Native coroutine: A coroutine function defined with `async def`. You can delegate from a native coroutine to another native coroutine using the `await` keyword, similar to how classic coroutines use `yield from`. The `async def` statement always defines a native coroutine, even if the `await` keyword is not used in its body. The `await` keyword cannot be used outside of a native coroutine.

- Classic coroutine: A generator function that consumes data sent to it via `my_coro.send(data)` calls, and reads that data by using `yield` in an expression. Classic coroutines can delegate to other classic coroutines using `yield from`. Classic coroutine cannot be driven by `await`, and are no longer supported by `asyncio`.

- Generator-based coroutine: A generator function decorated with `@types.coroutine` -- introduced in Python 3.5. That decorator makes the generator compatible with the new `await` keyword. It's in the process of being deprecated in favor of native coroutines.

- Asynchronous generator: A generator function defined with `async def` and using `yield` in its body. It returns an asynchronous generator object that provides `__anext__`, a coroutine method to retrieve the next item.

## An asyncio Example: Probing Domains

### Example 21-1. blogdom.py: search for domains for a Python blog

In [ ]:
import asyncio
import socket
from keyword import kwlist

MAX_KEYWORD_LEN = 4 #1

async def probe(domain: str) -> tuple[str, bool]: #2
    loop = asyncio.get_running_loop() #3
    try:
        await loop.getaddrinfo(domain, None) #4
    except socket.gaierror:
        return domain, False
    return domain, True

async def main(): #5
    names = (kw for kw in kwlist if len(kw) <= MAX_KEYWORD_LEN) #6
    domains = (f'{name}.dev'.lower() for name in names) #7
    coros = [probe(domain) for domain in domains] #8
    for coro in asyncio.as_completed(coros): #9
        domain, found = await coro  #10
        mark = '+' if found else '-' 
        print(f'{mark} {domain}')
        
if __name__ == '__main__':
    asyncio.run(main()) #11


1. Set maximum length of keyword for domains, because shorter is better.

2. `probe` returns a tuple with the domain name and a boolean; `True` means the domain resolved. Return the domain name will make it easier to display the results.

3. Get a reference to the `asyncio` event loop, so we can use it next.

4. The `loop.getaddrinfo()` coroutine-method returns a five-part tuple of parameters to connect to the given address using a socket. In this example, we don't need the result. If we got it, the domain resolves; otherwise, it doesn't.

5. `main` must be a coroutine, so that we can use `await` in it.

6. Generator to yield Python keywords with length to `MAX_KEYWORD_LEN`.

7. Generator to yield domains with the `.dev` suffix.

8. Build a list of coroutine objects by invoking the `probe` coroutine with each `domain` argument.

9. `asyncio.as_completed` is generator that yields coroutines that return the results of the coroutines passed to it in the order they are completed--not the order they were submitted. It's similar to `future.as_completed` in the previous chapter.

10. At this point, we know the coroutines is done because that's how `as_completed` works. Therefore, the `await` expression will not block but we need it to get the result from `coro`. If `coro` raised an unhandled exception, it would be re-raised here.

11. `asyncio.run` starts the event loop and return only when the event loop exits. This is a common pattern for scripts that use `asyncio`: implement `main` as a coroutine, and drive it with `asyncio.run` inside the `if __name__ == __main__:` block.

### Guido's Trick to Read Async Code

A trick suggested by Guido van Rossum himself: squint and pretend `async` and `await` keywords are not there. If you do that, you'll realize that coroutines read like plain old sequential functions.

## New Concept: Awaitable

The `await` keyword works with `awaitables`. As an end user of `asyncio`, these are the awaitables you'll encounter:

-  A `native coroutine object`, which you get by calling a `native coroutine function`.

- An `asyncio.Task`, which you usually get by passing a coroutine object to `asyncio.create_task()`.

## Downloading with asyncio and HTTPX

### Example 21-3. flags_asyncio.py: imports and download functions

In [3]:
import asyncio

from httpx import AsyncClient #1
from flags import BASE_URL, save_flag, main #2

async def download_one(client: AsyncClient, cc: str): #3
    image = await get_flag(client, cc) 
    save_flag(image, f'{cc}.gif')
    print(cc, end=' ', flush=True)
    return cc


async def get_flag(client: AsyncClient, cc: str) -> bytes: #4
    url = f'{BASE_URL}/{cc}/{cc}.gif'.lower() 
    resp = await client.get(url, timeout=6.1,
                            follow_redirects=True) #5
    return resp.read() #6

1. `httpx` must be installed--it's not part of the standard library.

2. Reuse code from `flags.py`.

3. `download_one` must be a native coroutine, so it can `await` on `get_flag`--which does the `HTTP` request. Then it displays the code of the downloaded flag, and saves the image.

4. `get_flag` needs to receive the `AsyncClient` to make the request.

5. The `get` method of an `httpx.AsyncClient` instance returns a `ClientResponse` object that is also an asynchronous context manager.

6. Network I/O operations are implemented as coroutine methods, so they are driven by the `asyncio` event loop.

### Example 21-2. flags_asynico.py: startup functions

In [ ]:
def download_many(cc_list: list[str]) -> int: #1
    return asyncio.run(supervisor(cc_list)) #2

async def supervisor(cc_list: list[str]) -> int: 
    async with AsyncClient() as client: #3
        to_do = [download_one(client, cc) 
                    for cc in sorted(cc_list)] #4
        res = await asyncio.gather(*to_do) #5
    return len(res) #6
    
if __name__ == '__main__':
    main(download_many) 

1. This needs to be a plain function--not a coroutine--so it can be passed to and called by the `main` function from the `main` function.

2. Execute the event loop driving the `supervisor(cc_list)` coroutine object until it returns. This will block while the event loop runs. The result of this line is whatever the `supervisor` coroutine returns.

3. `Asynchronous` HTTP client operations in `httpx` are methods of `AsyncClient`, which is also an asynchronous context manager: a context manager with asynchronous setup and teardown methods.

4. Build a list of coroutine objects by calling the `download_one` coroutine once for each flag to be retrieved.

5. Wait for the `asyncio.gather` coroutine, which accepts one or more awaitable arguments and waits for them to complete, returning a list of results for the given awaitables in the order they were submitted.

6. `supervisor` returns the length of the list returned by `asyncio.gather`.

asyncio.as_completed方法和asyncio.gather方法是Python asyncio库中用于并发执行协程的两种常用方法。它们在处理并发任务时有一些区别和特点。

1. asyncio.as_completed方法：
   - asyncio.as_completed方法接受一个协程列表作为参数，并返回一个可迭代对象。
   - 通过使用await关键字，可以逐个获取已完成的协程的结果。
   - 当有协程完成时，as_completed方法会立即返回该协程的结果，而不需要等待其他协程完成。
   - 这种方法适用于需要按照完成顺序逐个处理结果的情况。

2. asyncio.gather方法：
   - asyncio.gather方法接受一个或多个协程作为参数，并返回一个新的协程。
   - 通过使用await关键字，可以获取所有协程的结果。
   - gather方法会等待所有协程都完成后才返回结果，即所有协程都执行完毕后才会继续执行后续代码。
   - 这种方法适用于需要同时处理多个协程结果的情况。

总结：
- asyncio.as_completed方法适用于需要按照完成顺序逐个处理结果的情况，而asyncio.gather方法适用于需要同时处理多个协程结果的情况。
- asyncio.as_completed方法可以在协程完成时立即获取结果，而asyncio.gather方法需要等待所有协程完成后才返回结果。
- 根据具体需求选择合适的方法可以更好地处理并发任务。

### The Secret of Native Coroutine: Humble Generators

![asyncio event loop](./img/2023-11-24-14-33-22.png)

A key difference between the classic coroutine and native coroutine is that there are no visible `.send()` calls or `yield` expressions in the latter. This is illustrated in Figure 21-1.

Under the hood, the `asyncio` event loop makes the `.send()` calls that drive your coroutines, and your coroutines `await` on other coroutines, including library coroutines. As mentioned, `await` borrows most of implementation from `yield from`, which also makes `.send` calls to drive coroutines.

The `await` chain eventually reaches a low-level awaitable, which returns a generator that the event loop can drive in response to events such as timers or network I/O. The low-level awaitables and generators at the end of these `await` chains are implemented into the libraries, are not part of their APIs, and maybe Python/C extensions.

Using functions like `asyncio.gather` and `asyncio.created_task`, you can start multiple concurrent `await` channels, enabling concurrent execution of multiple I/O operations driven by a single event loop, in a single thread.

### The All-or-Nothing Problem

For peak performance with `asyncio`, we must replace every function that does I/O with an asynchronous with `asyncio`, we must replace every function that does I/O with an asynchronous version that is activated with `await` or `asyncio.create_task`, so that control is given back to the event loop while the function waits for I/O. If you can't rewrite a blocking function as a coroutine, you should run it in a separate thread or process.


How can you run a thread or process in a coroutine?

Short answer: use `ThreadPoolExecutor` and `as_completed` in `concurrent.futures` module.

## Asynchronous Context Managers

### Example 21-4. Sample code from documentation of the asyncpg PostgreSQL driver

```python
tr = connection.transaction()

await tr.start()
try:
    await connection.execute()
except:
    await tr.rollback()
    raise
else:
    await tr.commit()
```

A database transaction is a natural fit for the context manager protocol: the transaction has to be started, data is changed with `connection.execute`, and then a rollback or commit must happen, depending on the outcome of the operation.

In an asynchronous driver like `asyncpg`, the setup and wrap-up need to be coroutines so that other operations can happen concurrently. However, the implementation of the classic `with` statement doesn't support coroutines doing the work of `__enter__` and `__exit__`. 

That's why PEP 492--Coroutines with async and await syntax introduced the `async with` statement, which works with asynchronous context manager: objects implementing the `__aenter__` and `__aexit__` methods as coroutines.

With `async with`, Example 21-4 can be written like this other snippet from the `asyncpg` documentation:

```python
async with connection.transaction():
    await connection.execute("INSERT INTO mytable VALUES (1, 2, 3)")
```

In the `asyncpg.Transaction class`, the `__aenter__` coroutine method does `await self.start()`, and `__aexit__` does `await self.commit()` or `await self.rollback()` depending on whether an exception was raised in the `async with` block. Using coroutines to implement `Transaction` as an asynchronous context manager allows `asyncpg` to handle many transactions concurrently.

## Enhancing the asyncio downloader

## Delegating Tasks to Executors

## Writing asyncio Servers

## Asynchronous Iteration and Asynchronous Iterables

## async Beyond asyncio: Curio

## Type Hinting Asynchronous Objects

## How Async Works and How It doesn't

## Chapter Summary

## Further Reading